In [1]:
from transformers import pipeline,BertForMaskedLM
import torch
from transformers import BertTokenizer
from transformers import BertForMaskedLM
from transformers import AutoModelForMaskedLM
from transformers import TrainingArguments
# from data_processing import  get_all_data
from transformers import Trainer
from transformers import DataCollatorForLanguageModeling
from torch.utils.data import DataLoader
from transformers import default_data_collator
from torch.optim import AdamW
# from accelerate import Accelerator
from transformers import get_scheduler
from tqdm.auto import tqdm
import torch
import math
from transformers import AutoTokenizer
from datasets import load_dataset,DatasetDict,Dataset

/home/dlf/miniconda3/envs/gnn_ast_flow/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
dataset = load_dataset("csv",data_files="/home/dlf/prompt/dataset.csv")

Found cached dataset csv (/home/dlf/.cache/huggingface/datasets/csv/default-ac8894e355b6d41d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 422.56it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 66
    })
})

In [4]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "/home/dlf/prompt/code/model/bert_large_chinese"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


Some weights of the model checkpoint at /home/dlf/prompt/code/model/bert_large_chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def tokenize_function(examples):
    result = tokenizer(examples["text"],return_tensors="pt",padding="max_length",max_length=256)

    # for label in examples["label"]:
    #     print(label)
    #     print(tokenizer.convert_tokens_to_ids(str(label).strip().replace("\n","")))

    result["labels"] = [tokenizer.convert_tokens_to_ids(str(label).strip().replace("\n","")) for label in examples["label"]]
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text","label"]
)
tokenized_datasets
# print(tokenized_datasets["train"]["labels"])


Loading cached processed dataset at /home/dlf/.cache/huggingface/datasets/csv/default-ac8894e355b6d41d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-29f0a160e34e46ce.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 66
    })
})

In [11]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.0)

In [12]:
def group_texts(examples):
    result = {
        k: t
        for k, t in examples.items()
    }
    # Create a new labels column
    # 保存当前列的label
    label = result["labels"].copy()
    print(label)
    # 复制当前label过去
    result["labels"] = result["input_ids"].copy()
    
    for index,sentence_words in enumerate(result["input_ids"]):
        # 遍历当前句子的每一个word
        for word_index,word in enumerate(sentence_words):
            # 当前word 的id如果是等于mask的id
            if word == tokenizer.mask_token_id:
                # 第index个句子，第word_index个词的id为 = 第index个label
                result["labels"][index][word_index] = label[index]

    print(result["labels"] == result["input_ids"])
    return result

lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Loading cached processed dataset at /home/dlf/.cache/huggingface/datasets/csv/default-ac8894e355b6d41d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-142e99402da80929.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 66
    })
})

In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.00)

In [11]:
import collections
import numpy as np

from transformers import default_data_collator


def whole_word_masking_data_collator(features):
    # for feature in features:
    #     word_ids = feature.pop("word_ids")
 

    #     # Create a map between words and corresponding token indices
    #     mapping = collections.defaultdict(list)
    #     current_word_index = -1
    #     current_word = None
    #     for idx, word_id in enumerate(word_ids):
    #         if word_id is not None:
    #             if word_id != current_word:
    #                 current_word = word_id
    #                 current_word_index += 1
    #             mapping[current_word_index].append(idx)

    #     # Randomly mask words
    #     # mask = np.random.binomial(1, wwm_probability, (len(mapping),))
    #     mask = np.random.binomial(1, 0, (len(feature["input_ids"]),))
    #     # 添加mask位置
    #     for idx,word in enumerate(feature["input_ids"]):
    #         if word == tokenizer.mask_token_id:
    #             mask[idx] = 1

                
        # print(mask)
        # input_ids = feature["input_ids"]
        # labels = feature["labels"]
        # new_labels = [-100] * len(labels)
        # for word_id in np.where(mask)[0]:
        #     word_id = word_id.item()
        #     for idx in mapping[word_id]:
        #         new_labels[idx] = labels[idx]
        #         input_ids[idx] = tokenizer.mask_token_id
                
        # feature["labels"] = new_labels

    return default_data_collator(features)

# samples = [lm_datasets["train"][i] for i in range(2)]
# batch = whole_word_masking_data_collator(samples)

# for chunk in batch["input_ids"]:
#     print(f"\n'>>> {tokenizer.decode(chunk)}'")

In [12]:
train_size = 60
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)

downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 60
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 6
    })
})

In [13]:
from transformers import TrainingArguments

batch_size = 16
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    fp16=True,
    logging_steps=logging_steps,
)
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
import math

# eval_results = trainer.evaluate()
# print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [14]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [15]:


downsampled_dataset = downsampled_dataset.remove_columns(["word_ids","token_type_ids"])
# print(downsampled_dataset)
eval_dataset = downsampled_dataset["test"]

eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 6
})

In [16]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

In [17]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at /home/dlf/prompt/code/model/bert_large_chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [19]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [1]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        # break

    # Evaluation
    model.eval()
    losses = []
    # ⚠️ 这个地方为了测试过拟合用的是 训练集
    for step, batch in enumerate(train_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(loss)

    t_losses = torch.tensor(losses)
    t_losses = t_losses[: train_size]
    
    try:
        perplexity = math.exp(torch.mean(t_losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload


/home/dlf/miniconda3/envs/gnn_ast_flow/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'num_training_steps' is not defined

In [ ]:
mode.eval()
datas = [
    "在句子“脉细弱”中，词语“脉”的前文如果是由“[CLS]”词性的词语“[CLS]”来修饰，那么词语“脉”的词性是“[MASK]”",
    "在句子“脉细弱”中，词语“细”的前文如果是由“NR”词性的词语“脉”来修饰，那么词语“细”的词性是“[MASK]”",
    "在句子“脉细弱”中，词语“弱”的前文如果是由“VA”词性的词语“细”来修饰，那么词语“弱”的词性是“[MASK]”",
    "在句子“脉软”中，词语“脉”的前文如果是由“[CLS]”词性的词语“[CLS]”来修饰，那么词语“脉”的词性是“[MASK]”",
    "在句子“脉软”中，词语“软”的前文如果是由“NR”词性的词语“软”来修饰，那么词语“软”的词性是“[MASK]”",
    "在句子“脉细”中，词语“脉”的前文如果是由“[CLS]”词性的词语“[CLS]”来修饰，那么词语“脉”的词性是“[MASK]”,",
    "在句子“脉细”中，词语“细”的前文如果是由“NR”词性的词语“细”来修饰，那么词语“细”的词性是“[MASK]”",
]
with torch.no_grad():
    for item in datas:
        inputs = tokenizer(item, return_tensors="pt")
        print(inputs)
        token_logits = model(**inputs).logits
        mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
        mask_token_logits = token_logits[0, mask_token_index, :]
        top_5_tokens = torch.topk(mask_token_logits, 1, dim=1).indices[0].tolist()
        for token in top_5_tokens:
            print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")